In [ ]:
import inspect
import copy

In [ ]:
scores = [0, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [60]:
def parse_type(type_str):
    if type_str == "int":
        return int
    elif type_str == "str":
        return str
    elif type_str == "bool":
        return lambda x: x == "True"  # 字串轉 boolean
    elif type_str == "list[int]":
        return lambda x: list(map(int, x.strip().split()))
    elif type_str == "list[str]":
        return lambda x: x.strip().split()
    else:
        raise ValueError(f"Unsupported type: {type_str}")

def init(q=1, sample=True):
    if sample:
        testcase_num = 1
        fp = 'sample'
    else:
        testcase_num = 10
        fp = 'testcase'

    path = f'./testcase/Q{q}'
    with open(path + f'/Q{q}_info.txt') as file:
        info = file.readlines()
        info = [t.strip() for t in info]

        input_num = int(info[0])
        input_type = [parse_type(tp) for tp in info[1].split()]
        ans_type = [parse_type(tp) for tp in info[2].split()]

        params = []

        for i in range(1, testcase_num + 1):
            with open(path + f'/Q{q}_{fp}_{i:02d}.txt', 'r', encoding="utf-8") as f:
                text = [t.strip() for t in f.readlines()]
                inputs, ans = text[:input_num], text[input_num:]

                # 解析輸入
                parsed_inputs = [input_type[j](inputs[j]) for j in range(input_num)]

                # 解析輸出
                if len(ans_type) > 1:
                    parsed_ans = [ans_type[j](ans[j]) for j in range(len(ans_type))]
                else:
                    parsed_ans = ans_type[0]('\n'.join(ans))  # 多行輸出轉成單一物件

                params.append({'input': parsed_inputs, 'output': parsed_ans})

    return params

In [ ]:
def map_input(i, var_names, var_vals, ans):
    hint_str = f"【提示】\n第 {i} 筆測試資料：\n輸入：{var_names[0]} = {var_vals[0]}"
    for i in range(1, len(var_names)):
        hint_str += (f", {var_names[i]} = {var_vals[i]}")
    hint_str += f"\n正確答案： "
    hint_str += f"{ans}"
    return hint_str

In [ ]:
class Problem:
    def __init__(self, id):
        self.id = id
        self.sample = init(self.id, True)
        self.__testcase = init(self.id, False)
        self.score = 0

    def check_sample(self, func):
        for i, case in enumerate(self.sample):
            input = case['input']
            ans = case['output']
            written_ans = func(*copy.deepcopy(input))
            if(written_ans == ans):
                # print(f'Sample {i + 1} Passed')
                print("範例測試通過！")
            else:
                # print(f'Sample {i + 1} Failed')
                print("\033[91m範例測試失敗！\033[0m")

    def check_ans(self, func):
        score = 0
        hint_str = ""
        for i, case in enumerate(self.__testcase):
            input = case['input']
            ans = case['output']
            written_ans = func(*copy.deepcopy(input))
            if(written_ans == ans):
                score += scores[self.id]
                print(f'第 {i + 1} 筆測試資料通過')
            else:
                if(hint_str == ""):
                    hint_str = map_input(i + 1, inspect.getfullargspec(func).args, input, ans)
                print(f'\033[91m第 {i + 1} 筆測試資料失敗\033[0m')
        print(f"\n得分：{score}\n")
        if(score != 20):
            print(f"{hint_str}")
        self.score = max(self.score, score)


In [64]:
problem = [Problem(i) for i in range(1, 10)]
problem.insert(0, 0)

In [65]:
def view_score():
    print(f"得分：{sum([problem[i].score for i in range(1, 7)])}")